# Example: Net Present Value for a Tesla Model S
In this example, we'll compute the [Net Present Value (NPV)](https://en.wikipedia.org/wiki/Net_present_value) for purchasing, owning and selling a [2024 Tesla Model S](https://www.tesla.com/models). In particular, using the Abstract Asset approach, we'll compute the cash flow events associated with purchasing, owning, and selling the [Model S](https://www.tesla.com/models). Is purchasing a [Model S](https://www.tesla.com/models) a good investment? I think we should find out.

### Learning objectives
This example will familiarize students with computing the [Net Present Value (NPV)](https://en.wikipedia.org/wiki/Net_present_value), i.e., stipulating the net cash flow events, computing the discount factors for different periods and ultimately computing the net present value.
* __Prerequisite__: Setup problem components and constants. Before we do the [Net Present Value (NPV) calculation](https://en.wikipedia.org/wiki/Net_present_value#:~:text=NPV%20is%20the%20sum%20of,NPV%20results%20in%20a%20loss.), we'll set up some components and constants that we'll use later.
* __Task 1__: Setup the Cash Flow Dictionary. In this task, we specify some values for the cash flow events we observe over the lifetime of owning the [Tesla Model S](https://www.tesla.com/models). Then, we construct a net cash flow dictionary that holds the net values for the cash flow events during each period.
* __Task 2__: In this task, we compute a dictionary of discount factor $\mathcal{D}_{j,0}(\bar{r})$ values using the time and discount rate data specified in the Prerequisites section.
* __Task 3__: In this task, we'll compute the net present value (NPV) for the [Tesla Model S](https://www.tesla.com/models) and answer some disu

## Setup
Set up the computational environment using the `Include.jl` file. The `Include.jl` file loads external packages, various functions we will use in the exercise, and custom types to model the components of our example problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

In [3]:
include("Include.jl")

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
    Updating `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-1/L1b/Project.toml`
  [4119e0bf] ~ VLQuantitativeFinancePackage v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main` ⇒ v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main`
    Updating `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-1/L1b/Manifest.toml`
  [4119e0bf] ~ VLQuantitativeFinancePackage v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main` ⇒ v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main`
Precompiling project...
  ✓ VLQuantitativeFinancePackage
  1 dependency successfully precompiled in 4 seconds. 340 already precompiled.
  Activating project at `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-1/L1b

##  Prerequisite: Setup problem components and constants
Before we do the [Net Present Value (NPV) calculation](https://en.wikipedia.org/wiki/Net_present_value#:~:text=NPV%20is%20the%20sum%20of,NPV%20results%20in%20a%20loss.), we'll set up some components and constants that we'll use later. 
* Let's start with the discounting model. For simplicity, we'll assume discrete compounding in this calculation. We specify this by creating an instance of [the `DiscreteCompoundingModel` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.DiscreteCompoundingModel) and save it in the `discount_model::DiscreteCompoundingModel` variable.

In [5]:
discount_model = DiscreteCompoundingModel();

Next, we set several constants, which we use below. For a description of the constant and permissible values, see the comment string beside the declaration.

In [7]:
T = 2.0;  # number of years; value ≥ 0
λ = 2.0 |> Int;  # number of periods per year (λ = 2.0 => every six months); value ≥ 1
r̄ = 0.04; # discount rate per year; value ≥ 0
depreciation = 0.03; # price decline per period
N = λ*T |> Int;  # total number of observations

## Task 1: Setup the Cash Flow Dictionary
In this task, we specify some values for the cash flow events we observe over the lifetime of owning the [Tesla Model S](https://www.tesla.com/models). Then, we construct a net cash flow dictionary that holds the net values for the cash flow events during each period. Let's start with the constants.
* Specify the purchase price, sale price at the end of $T$-years, insurance costs per period, and other costs and savings below. For a description of the constant and permissible values, see the comment string beside the declaration.

In [9]:
purchase_price = 50000.0; # how much do we pay now for the Model S?
sale_price = (1 - N*depreciation)*purchase_price; # assumes straight line depreciation
insurance_costs = 1808.0; # How much does a Model S cost to ensure (per period)?
other_costs = 50.0; # Other costs associated with the Model S per period
other_savings = 0.0; # Other savings per period?

Now, let's build and populate the cash flow event dictionary. The dictionary will hold the net cash flow events for each $N = \lambda\cdot{T}$ period. We'll populate this dictionary by iterating over the `0...N` periods using a [Julia `for-loop`](https://docs.julialang.org/en/v1/base/base/#for).
* For each period (index `i`), we check if we are `i == 0` (purchase), or `i == N` (sale) of the car. At purchase time, we have a negative cash flow (we give a bunch of money to [TSLA](https://finance.yahoo.com/quote/TSLA/?guccounter=1&guce_referrer=aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8&guce_referrer_sig=AQAAANiorcaltDNUCtp01qZd0GAF86DY9ajVMBq8fpt9wPQJKA2M7XgMOWvsfeDsrJxSlpMUKrWYfPDaC9XhnhQ2uCQnfWqvJIdpkimycwOTPv_QySkAzvnIju10jMrqSK7KdsX4ATgr0VtRlF6up-NuK5jDvJx1v2NSlxu1RGQTCAxR)) and at sale time we have the sale price (positive) and any other savings minus the expenses for that period.
* If $i\neq{0}\,||\,i\neq{N}$: we have an intermediate period where we make an insurance payment net any other savings occurring during that period.
* The keys of the `cash_flow_event_dictionary` are the period indexes `i`, while the values are the net cash flow events for that period.

In [11]:
cash_flow_event_dictionary = Dict{Int64,Float64}();
for i ∈ 0:N
    if (i == 0)
       cash_flow_event_dictionary[i] = -1*purchase_price;
    elseif (i == N)
        cash_flow_event_dictionary[i] = sale_price + other_savings - (insurance_costs+other_costs)
    else
       cash_flow_event_dictionary[i] = other_savings - (insurance_costs+other_costs);
    end
end
cash_flow_event_dictionary

Dict{Int64, Float64} with 5 entries:
  0 => -50000.0
  4 => 42142.0
  2 => -1858.0
  3 => -1858.0
  1 => -1858.0

## Task 2: Compute the Discount Factor Dictionary
In this task, we compute a dictionary of discount factor $\mathcal{D}_{j,0}(\bar{r})$ values using the time and discount rate data specified above. To do this calculation, we call [the `discount(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.discount-Tuple{AbstractCompoundingModel,%20Float64,%20Int64}) which returns the discount factors in the `discount_dictionary::Dict{Int64,Float64}` dictionary.
* The [`discount(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.discount-Tuple{AbstractCompoundingModel,%20Float64,%20Int64}) takes the compounding model instance, i.e., the `discount_model` instance we constructed earlier, along with the discount rate $\bar{r}$, the number of periods $N$, and the number of compounding events per period $λ$.
* _What is up with the $\lambda$ arg_? We implement the compounding events per period $\lambda$ as a [optional keyword argument](https://docs.julialang.org/en/v1/manual/functions/#Keyword-Arguments) with a default value of $\lambda = 2$ in [the `discount(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.discount-Tuple{AbstractCompoundingModel,%20Float64,%20Int64}). 

In [13]:
discount_dictionary = discount(discount_model, r̄, N, λ = λ);

`Unhide` the code block below to see how we built a table of the data in the `discount_dictionary` using [the `pretty_table(...)` function exported by the PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl) and [a `DataFrame` instance exported by the DataFrames.jl package](https://github.com/JuliaData/DataFrames.jl)

In [42]:
let
    # initialize -
    df = DataFrame();
    for i ∈ 0:N
        value = discount_dictionary[i];
        row_df = (
            period = i,
            𝒟 = value,
            𝒟inv = (1/value)
        );
        push!(df, row_df);
    end

    pretty_table(df, tf = tf_simple)
end

========= ========= ===========
  period         𝒟       𝒟inv 
   Int64   Float64    Float64 
========= ========= ===========
       0       1.0        1.0
       1      1.02   0.980392
       2    1.0404   0.961169
       3   1.06121   0.942322
       4   1.08243   0.923845
========= ========= ===========


### Check: Do we recover the discount rate $\bar{r}$?
Let's do a quick check on the discount factor calculation. If the discount factors are correct, we should be able to recover the discount rate $\bar{r}$ by inverting the definition of the discount factor, i.e., the discount rate should be equal to:
$$
\begin{equation}
\bar{r} = \lambda\cdot\left(\mathcal{D}_{i,0}^{1/i} - 1\right)
\end{equation}
$$
where $\mathcal{D}_{i,0}$ is the discount factor, $i$ is the period-index, and $\lambda$ is the number of compounding events per-period. This is the value for the period $i\geq{1}$. 
* We test this idea by iterating over the `discount_dictionary,` and for iteration $i>0$, we compute the discount rate (stored in the `r̄ᵢ` variable) and compare its value for the specified (correct) discount rate `r̄.` We use [the Julia `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with [the `isapprox(...)` method](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to check for equality. If the check fails, [an AssertionError is thrown](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) and the loop terminates.

In [52]:
r̄ᵢ = r̄;
for (k,v) ∈ discount_dictionary
    if (k > 0)
        r̄ᵢ = λ*((v)^(1/k) - 1);
    end
    @assert isapprox(r̄, r̄ᵢ, rtol = 1e-4);
end

## Task 3: Compute the Net Present Value (NPV)
In this task, we'll compute the net present value (NPV) for the [Tesla Model S](https://www.tesla.com/models). The NPV is the scalar (dot) product between the inverse of the discount factor vector $\mathcal{D}^{-1}$ and the net cash flow vector $\bar{c}$:
$$
\begin{equation}
\texttt{NPV} \leftarrow \left<\mathcal{D}^{-1}\left(\bar{r}\right), \bar{c}, \right>
\end{equation}
$$
However, we have one technical gotcha: the discount factors and the net cash flow events are stored in dictionaries, so we'll first need to convert this data into arrays and then use [the `dot(...)` method exported by the LinearAlgebra.jl package](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.dot) to compute the scalar product. Let's start with the discount factors and the we'll convert the cash flow events.

#### Compute the inverse discount factor array $\mathcal{D}^{-1}(\bar{r})$
To compute the inverse discount factor array $\mathcal{D}^{-1}(\bar{r})$, we iterate over the periods `i ∈ 1...N+1` (where we convert to `1`-based indexing). We insert the inverse of the value in the `discount_dictionary` for each period `i` into the `𝒟inv::Array{Float64,1}` array.

In [55]:
𝒟inv = Array{Float64,1}(undef, N+1);
for i in 1:(N+1)
    j = i - 1;
    𝒟inv[i] = 1/discount_dictionary[j]
end
𝒟inv;

#### Compute the cash flow event array $\bar{c}$.
To compute the cash flow event array $\bar{c}$, we iterate over the periods `i ∈ 1...N+1` (where we convert to `1`-based indexing). We insert the value from the `cash_flow_event_dictionary` for each period `i` into the `c̄::Array{Float64,1}` array.

In [94]:
c̄ = Array{Float64,1}(undef, N+1);
for i ∈ 1:(N+1)
    j = i - 1;
    c̄[i] = cash_flow_event_dictionary[j]
end
c̄;

Finally, we compute the $\texttt{NPV}$ using [the `dot(...)` method exported by the LinearAlgebra.jl package](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.dot).

In [111]:
NPV = dot(𝒟inv, c̄);
println("The NPV for a Tesla Model S over $(T) years equals $(NPV) USD")

The NPV for a Tesla Model S over 2.0 years equals -16425.5611769702 USD


### Discussion Questions
* What does having a negative `NPV < 0` value mean?
* What factors can we change to increase the `NPV` of the [Tesla Model S](https://www.tesla.com/models)?

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.